In [ ]:
import pandas as pd
amazon = pd.read_csv(r"C:\Users\Administrator\Desktop\PADA_LAB\final_data\amazon_sentiment.csv", encoding='latin1')
audible = pd.read_csv(r"C:\Users\Administrator\Desktop\PADA_LAB\final_data\audible_sentiment.csv")
coursera = pd.read_csv(r"C:\Users\Administrator\Desktop\PADA_LAB\final_data\coursera_sentiment.csv")
hotel = pd.read_csv(r"C:\Users\Administrator\Desktop\PADA_LAB\final_data\hotel_sentiment.csv")

In [ ]:
!pip install ydata_profiling
#인터렉션 데이터 분포 확인용용
import pandas as pd
from ydata_profiling import ProfileReport

# 프로파일링 리포트 생성 (수정된 파일 기반)
profile = ProfileReport(amazon, title="Profiling Report")
profile.to_file("distribution_check_amazon.html")

## 전처리 코드(스케일링 포함)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
def preprocess(df, scaling_name):
    removed_columns = df.select_dtypes(include=['object']).columns.tolist()
    removed_columns.append('Rating')
    print(removed_columns)
    
    df_clean = df.drop(columns=removed_columns)
    
    if scaling_name =='standard':
        scaler = StandardScaler()
    elif scaling_name == 'minmax':
        scaler = MinMaxScaler()
    elif scaling_name == 'robust':
        scaler = RobustScaler()
    
    df_scaled = pd.DataFrame(scaler.fit_transform(df_clean), columns=df_clean.columns)
    return df_scaled
    
amazon_scaled = preprocess(amazon, 'standard')
audible_scaled = preprocess(audible, 'standard')
coursera_scaled = preprocess(coursera, 'standard')
hotel_scaled = preprocess(hotel, 'standard')

['Product_Name', 'Review_Title', 'Review_Text', '']
['Review_Text', 'Posted_Date', 'review_title', 'Rating_Distribution', 'Rating_of_Performance', 'Rating_of_Story', 'Crawled_date', 'Product_Name', 'Categories', '']
['course_name', 'Review_Title', '']
['Hotel_Name', 'Review_Text', 'Review_Title', 'Crawled_Date', '']
